## CODE FOR VISUALISING BOROUGHS AROUND TORONTO AND EXPLORING MEXICAN RESTAURANT LOCATED CLOSE USING FOURSQUARE API

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
import matplotlib.pyplot as plt
from unicodedata import normalize
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import requests
page = requests.get(url)
page

<Response [200]>

In [3]:
page.status_code

200

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)



In [6]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})



In [7]:
df.shape

(103, 3)

In [8]:
df2=pd.read_csv('Geospatial_Coordinates.csv')
df2.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
Toronto = pd.merge(df, df2, on ='PostalCode')
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'],Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto





In [25]:
#Let's define my FourSquare Credentials

CLIENT_ID = 'PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ' # your Foursquare ID
CLIENT_SECRET = 'F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW' # your Foursquare Secret
ACCESS_TOKEN = '1XWQ3JFAXHVWDJ3WFUHS0VC0SKXTGEFIZGHXAD5CKPZWAVVZ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ
CLIENT_SECRET:F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW


In [26]:
#Let us search for a restaurant around the Toronto boroughs where we can get a Mexican Cuisine
search_query = 'Mexican'
radius = 500
print(search_query + ' .... OK!')

Mexican .... OK!


In [27]:
#Define the url that will cull the information from Foursquare
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ&client_secret=F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW&ll=43.6534817,-79.3839347&oauth_token=1XWQ3JFAXHVWDJ3WFUHS0VC0SKXTGEFIZGHXAD5CKPZWAVVZ&v=20180604&query=Mexican&radius=500&limit=30'

In [28]:
#send a GET request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607967be6c881064dfa70a71'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4adc9148f964a520512d21e3',
    'name': 'Chipotle Mexican Grill',
    'location': {'address': '323 Yonge St, Unit 114',
     'crossStreet': 'Yonge & Dundas',
     'lat': 43.65686,
     'lng': -79.38091,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65686,
       'lng': -79.38091}],
     'distance': 448,
     'postalCode': 'M5B 1R7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['323 Yonge St, Unit 114 (Yonge & Dundas)',
      'Toronto ON M5B 1R7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1c1941735',
      'name': 'Mexican Restaurant',
      'pluralName': 'Mexican Restaurants',
      'shortName': 'Mexican',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
       'suffix': '.png'},
  

In [31]:
from pandas.io.json import json_normalize
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe_Toronto = json_normalize(venues)
dataframe_Toronto.head()

C:\Users\BEDC\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4adc9148f964a520512d21e3,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1618569150,False,"323 Yonge St, Unit 114",Yonge & Dundas,43.656860,-79.380910,"[{'label': 'display', 'lat': 43.65686, 'lng': ...",448,M5B 1R7,CA,Toronto,ON,Canada,"[323 Yonge St, Unit 114 (Yonge & Dundas), Toro..."
1,4f7d9a0ee4b01f9f43bd0603,Mexican Consulate,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1618569150,False,11 King St,NaN,43.648942,-79.378676,"[{'label': 'display', 'lat': 43.64894155562399...",659,NaN,CA,Toronto,ON,Canada,"[11 King St, Toronto ON, Canada]"
2,4d76c9eec95f5941ca6e2321,Chimichanga Mexican Grill,[],v-1618569150,False,70 Yorkville Ave,Yorkville/Bel Air,43.651772,-79.379482,"[{'label': 'display', 'lat': 43.65177154540889...",406,M5R 1B9,CA,Toronto,ON,Canada,"[70 Yorkville Ave (Yorkville/Bel Air), Toronto..."
3,4bdef9836198c9b6de8115ff,Mucho Burrito Fresh Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1618569150,False,333 Bay St.,Bay Adelaide Centre,43.650661,-79.379401,"[{'label': 'display', 'lat': 43.65066054666582...",481,M5H 2R2,CA,Toronto,ON,Canada,"[333 Bay St. (Bay Adelaide Centre), Toronto ON..."
4,4f0240df6da1cd035b2227eb,Mucho Burrito Fresh Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1618569150,False,"263 Queen St W,",btw Duncan & St Patrick,43.650158,-79.389157,"[{'label': 'display', 'lat': 43.650158, 'lng':...",560,M5V 1Z4,CA,Toronto,ON,Canada,"[263 Queen St W, (btw Duncan & St Patrick), To..."


In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_Toronto.columns if col.startswith('location.')] + ['id']
dataframe_filtered_Toronto = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_Toronto['categories'] = dataframe_filtered_Toronto.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_Toronto.columns = [column.split('.')[-1] for column in dataframe_filtered_Toronto.columns]

dataframe_filtered_Toronto.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Chipotle Mexican Grill,Mexican Restaurant,"323 Yonge St, Unit 114",Yonge & Dundas,43.656860,-79.380910,"[{'label': 'display', 'lat': 43.65686, 'lng': ...",448,M5B 1R7,CA,Toronto,ON,Canada,"[323 Yonge St, Unit 114 (Yonge & Dundas), Toro...",4adc9148f964a520512d21e3
1,Mexican Consulate,Embassy / Consulate,11 King St,NaN,43.648942,-79.378676,"[{'label': 'display', 'lat': 43.64894155562399...",659,NaN,CA,Toronto,ON,Canada,"[11 King St, Toronto ON, Canada]",4f7d9a0ee4b01f9f43bd0603
2,Chimichanga Mexican Grill,None,70 Yorkville Ave,Yorkville/Bel Air,43.651772,-79.379482,"[{'label': 'display', 'lat': 43.65177154540889...",406,M5R 1B9,CA,Toronto,ON,Canada,"[70 Yorkville Ave (Yorkville/Bel Air), Toronto...",4d76c9eec95f5941ca6e2321
3,Mucho Burrito Fresh Mexican Grill,Mexican Restaurant,333 Bay St.,Bay Adelaide Centre,43.650661,-79.379401,"[{'label': 'display', 'lat': 43.65066054666582...",481,M5H 2R2,CA,Toronto,ON,Canada,"[333 Bay St. (Bay Adelaide Centre), Toronto ON...",4bdef9836198c9b6de8115ff
4,Mucho Burrito Fresh Mexican Grill,Mexican Restaurant,"263 Queen St W,",btw Duncan & St Patrick,43.650158,-79.389157,"[{'label': 'display', 'lat': 43.650158, 'lng':...",560,M5V 1Z4,CA,Toronto,ON,Canada,"[263 Queen St W, (btw Duncan & St Patrick), To...",4f0240df6da1cd035b2227eb


In [33]:
#Visualise the Mexican Restaurant in close proximity
dataframe_filtered_Toronto.name

0               Chipotle Mexican Grill
1                    Mexican Consulate
2            Chimichanga Mexican Grill
3    Mucho Burrito Fresh Mexican Grill
4    Mucho Burrito Fresh Mexican Grill
Name: name, dtype: object

In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Toronto showing the Mexican restaurants

# add a red circle marker to represent Toronto
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Mexican restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_Toronto.lat, dataframe_filtered_Toronto.lng, dataframe_filtered_Toronto.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map